In [73]:
import numpy as np
import pandas as pd

In [74]:
np.random.seed(23)

In [75]:
mu_vec1 = np.array([0,0,0])
cov_mat1 = np.array([[1,0,0],[0,1,0],[0,0,1]])
class1_sample = np.random.multivariate_normal(mu_vec1, cov_mat1, 20)

In [76]:
df = pd.DataFrame(class1_sample, columns=['f1', 'f2', 'f3'])
df['target'] = 1

In [77]:
mu_vec2 = np.array([1,1,1])
cov_mat2 = np.array([[1,0,0],[0,1,0],[0,0,1]])
class2_sample = np.random.multivariate_normal(mu_vec2, cov_mat2, 20)

In [78]:
df1 = pd.DataFrame(class2_sample, columns=['f1', 'f2', 'f3'])
df1['target'] = 0

In [79]:
df = pd.concat([df, df1], axis=0)

In [80]:
df = df.sample(20)

In [81]:
df

,f1,f2,f3,target
2,-0.367548,-1.137460,-1.322148,1
14,0.177061,-0.598109,1.226512,0
14,0.420623,0.411620,-0.071324,1
11,1.968435,-0.547788,-0.679418,1
12,-2.506230,0.146960,0.606195,1
9,1.425140,1.441152,0.182561,0
11,2.224431,0.230401,1.192120,0
4,0.322272,0.060343,-1.043450,1
12,-0.723253,1.461259,-0.085367,0
13,2.823378,-0.332863,2.637391,0


In [82]:
import plotly.express as px

In [83]:
fig = px.scatter_3d(df, x='f1', y='f2', z='f3', color=df['target'].astype('str'))
fig.update_traces(marker=dict(size=12, line=dict(width=2, color='DarkSlateGrey')), selector=dict(mode='markers'))
fig.show()

In [84]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [85]:
df.iloc[:,0:3] = scaler.fit_transform(df.iloc[:,0:3])

In [86]:
import numpy as np

covariance_matrix = np.cov(df.iloc[:, [0, 1, 2]].T)
covariance_matrix

array([[ 1.05263158,  0.18345809,  0.09157879],
       [ 0.18345809,  1.05263158, -0.03936926],
       [ 0.09157879, -0.03936926,  1.05263158]])

In [87]:
# finding eigen vectors and eigen values
eigen_values, eigen_vectors = np.linalg.eig(covariance_matrix)

In [88]:
eigen_values

array([0.83006218, 1.24416981, 1.08366274])

In [89]:
eigen_vectors

array([[-0.67597553,  0.72707359,  0.12008781],
       [ 0.62597306,  0.65251607, -0.42706031],
       [ 0.3888635 ,  0.21351059,  0.89621337]])

In [90]:
%pylab inline 

from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import axis3d
from mpl_toolkits.mplot3d import proj3d
from matplotlib.patches import FancyArrowPatch

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [91]:
class Arrow3d(FancyArrowPatch):
    def __init__(self, xs, ys, zs, *args, **kwargs):
        FancyArrowPatch.__init__(self, (0,0), (0,0), *args, **kwargs)
        self._verts3d = xs, ys, zs

    def draw(self, renderer):
        xs3d, ys3d, zs3d = self._verts3d
        xs, ys, zs = proj3d.proj_transform(xs3d, ys3d, zs3d, renderer.M)
        self.set_positions((xs[0], ys[0]), (xs[1], ys[1]))
        FancyArrowPatch.draw(self, renderer)

In [92]:
fig = plt.figure(figsize=(7,7))
ax = fig.add_subplot(111, projection='3d')

ax.plot(df['f1'], df['f2'], df['f3'], 'o', markersize=8, color='green', alpha=0.2)
ax.plot(df['f1'].mean(), df['f2'].mean(), df['f3'].mean(), 'o', markersize=10, color='red', alpha=0.5)
for v in eigen_vectors.T:
    a = Arrow3d([df['f1'].mean(), v[0]], [df['f2'].mean(), v[1]], [df['f3'].mean(), v[2]], mutation_scale=20, lw=3, arrowstyle='-|>', color='r')
    ax.add_artist(a)
ax.set_xlabel('f1')
ax.set_ylabel('f2')
ax.set_zlabel('f3')

plt.title('Eigenvectors')
plt.show()

AttributeError: 'Arrow3d' object has no attribute 'do_3d_projection'

<Figure size 700x700 with 1 Axes>

In [93]:
pc = eigen_vectors[0:2]
pc

array([[-0.67597553,  0.72707359,  0.12008781],
       [ 0.62597306,  0.65251607, -0.42706031]])

In [94]:
transformed_df = np.dot(df.iloc[:, [0, 1, 2]], pc.T)

new_df = pd.DataFrame(transformed_df, columns=['PC1', 'PC2'])
new_df['target'] = df['target'].values
new_df.head()

,PC1,PC2,target
0,-0.743811,-0.383050,1
1,-0.383296,-0.980503,0
2,-0.073047,0.262240,1
3,-1.529062,0.770653,1
4,1.381386,-1.598120,1


In [95]:
new_df['target'] = new_df['target'].astype('str')
fig = px.scatter(new_df, x='PC1', y='PC2', color='target', color_discrete_sequence=px.colors.qualitative.Set1)

fig.update_traces(marker=dict(size=12, line=dict(width=2, color='DarkSlateGrey')), selector=dict(mode='markers'))
fig.show()